__Business Statement__: The prod and delivery files contain a column called "Units Adjusted by OIMA." The OIMA reviews water reports to determine if the units used in the original report are correct. OIMA manually reviews the water reports to look for deviations from average gallons per capita day and PWS trends. If they believe the original units are incorrect they present revised units. 

__Objective__: Understand how often prod and delivery reports are submitted in the incorrect units.

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.width', 5000) 
pd.set_option('display.max_colwidth', -1) 
pd.set_option('display.max_columns', 60)

In [3]:
processed_data_path  ="../data/interim/"

### Read in prod and delivery files in state before we convert units to gallons using revised units

In [4]:
prod_processed = pd.read_csv("{}1_initial_eda___prod_before_unit_conversion.csv".format(processed_data_path))
deliv_processed = pd.read_csv("{}1_initial_eda___deliv_before_unit_conversion.csv".format(processed_data_path))

In [5]:
deliv_processed['Date'] = pd.to_datetime(deliv_processed['Date'])
prod_processed['Date'] = pd.to_datetime(prod_processed['Date'])

#### Create a 0/1 flag indicating if units are incorrect or unavailable

In [6]:
prod_processed['original_units_incorrect'] = prod_processed[['WATER PRODUCED Water.Units IN UNITS ORIGINALLY REPORTED',
                                                            'WATER PRODUCED Water.Units REVIEWED BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS']].apply(lambda x: 1 if x[0]!= x[1] else 0,axis=1)
deliv_processed['original_units_incorrect'] = deliv_processed[['Delivered.Water.Units AS ORIGINALLY REPORTED',
                                                            'Delivered.Water.Units.Revised BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS']].apply(lambda x: 1 if x[0]!= x[1] else 0,axis=1)

prod_processed['units_unavailable'] = prod_processed['WATER PRODUCED Water.Units REVIEWED BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS'].apply(lambda x: 1 if x=="-" else 0)
deliv_processed['units_unavailable'] = deliv_processed[ 'Delivered.Water.Units.Revised BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS'].apply(lambda x: 1 if x=="-" else 0)

### How often are units reported incorrectly?

In [7]:
prod_processed[prod_processed['units_unavailable']==0]['original_units_incorrect'].value_counts() / prod_processed[prod_processed['units_unavailable']==0].shape[0]

0    0.974498
1    0.025502
Name: original_units_incorrect, dtype: float64

In [8]:
# What % of records have units reported incorrectly?
deliv_processed[deliv_processed['units_unavailable']==0]['original_units_incorrect'].value_counts() / deliv_processed[deliv_processed['units_unavailable']==0].shape[0]

0    0.933116
1    0.066884
Name: original_units_incorrect, dtype: float64

The delivery file appears to have more issues with incorrect units that the prod file.

### Are there trends over time?

In [9]:
incorrectByYear = deliv_processed[deliv_processed['units_unavailable']==0][['Year','original_units_incorrect']].groupby('Year').agg({'original_units_incorrect':['sum','count']}).reset_index()
incorrectByYear['pct'] = incorrectByYear[('original_units_incorrect', 'sum')] / incorrectByYear[('original_units_incorrect', 'count')]
incorrectByYear

Year original_units_incorrect              pct
                             sum  count          
0  2013  1227                     15494  0.079192
1  2014  1637                     17153  0.095435
2  2015  1517                     21772  0.069677
3  2016  819                      23328  0.035108

In [10]:
incorrectByYear = prod_processed[prod_processed['units_unavailable']==0][['Year','original_units_incorrect']].groupby('Year').agg({'original_units_incorrect':['sum','count']}).reset_index()
incorrectByYear['pct'] = incorrectByYear[('original_units_incorrect', 'sum')] / incorrectByYear[('original_units_incorrect', 'count')]
incorrectByYear

Year original_units_incorrect              pct
                             sum  count          
0  2013  2012                     43070  0.046715
1  2014  2394                     53149  0.045043
2  2015  546                      54560  0.010007
3  2016  461                      61476  0.007499

The delivery file reached a low of 3.5% incorrect reports in 2016, vs. 6-9% in prior years. However, Since close to 24K reports were filed in 2016, we can explore using modeling to identify incorrect units. This would save staff the time of manually reviewing reports.

### Are there water sytems that reported incorrectly multiple times?

In [11]:
firstLastTimeIncorrect =  deliv_processed[ (deliv_processed['units_unavailable']==0) & (deliv_processed['original_units_incorrect']==1)]\
[['Water.System.Name','Date']].groupby('Water.System.Name').agg({"Date": [min,max]}).reset_index()
firstLastTimeIncorrect[0:10]

Water.System.Name       Date           
                                                   min        max
0  AERIAL ACRES WATER SYSTEM                2013-01-01 2014-12-01
1  AFTON STORE                              2016-01-01 2016-12-01
2  ALDERCROFT HEIGHTS COUNTY WATER DISTRICT 2014-01-01 2014-12-01
3  ALPINE ACRES MUTUAL WATER CO             2013-03-01 2013-12-01
4  ANDERSON SPRINGS CSD                     2014-01-01 2015-12-01
5  APTCO LLC                                2014-01-01 2015-12-01
6  AROMAS HILLS MUTUAL WATER ASSOCIATION    2013-01-01 2013-12-01
7  ASHFORD HIGHLANDS MWC                    2015-01-01 2015-12-01
8  ASOLEADO MWC                             2015-02-01 2015-12-01
9  BADGER HILL ESTATES                      2015-01-01 2015-12-01

In [12]:
# 142 systems were incorrect for first time in Jan 2014; last time in Dec 2014
# 123 systems were incorrect for first time in Jan 2013; last time in Dec 2013
firstLastTimeIncorrect.groupby([ ('Date', 'min'), ('Date', 'max')]).count().reset_index().sort_values('Water.System.Name',ascending=False)[0:10]

Date            Water.System.Name
          min        max                  
13 2014-01-01 2014-12-01  77              
26 2015-01-01 2015-12-01  66              
1  2013-01-01 2013-12-01  61              
38 2016-01-01 2016-12-01  27              
5  2013-01-01 2016-12-01  15              
2  2013-01-01 2014-12-01  13              
14 2014-01-01 2015-12-01  11              
15 2014-01-01 2016-12-01  11              
27 2015-01-01 2016-12-01  10              
3  2013-01-01 2015-12-01  9

In [13]:
# generally systems are incorrect for 12 months
numTimesIncorrectBySystem = deliv_processed[(deliv_processed['units_unavailable']==0) & (deliv_processed['original_units_incorrect']==1)][['Water.System.Name','original_units_incorrect']].groupby('Water.System.Name').sum().reset_index()
numTimesIncorrectBySystem['original_units_incorrect'].value_counts().reset_index()[0:10]

,index,original_units_incorrect
0,12,231
1,24,42
2,6,16
3,36,12
4,48,11
5,4,7
6,8,5
7,1,5
8,18,4
9,9,4


### In each FY, How many water systems deliver their first report? 
Systems might be more likely to misreport units on their first report

In [14]:
earliestProd = prod_processed[['Water.System.Name','Date']].groupby(["Water.System.Name"]).min().reset_index()
earliestDeliv = deliv_processed[['Water.System.Name','Date']].groupby(["Water.System.Name"]).min().reset_index()

In [15]:
earliestProd['Date'].value_counts().reset_index()[0:10]

,index,Date
0,2013-01-01,3469
1,2014-01-01,979
2,2016-01-01,744
3,2015-01-01,520
4,2013-05-01,73
5,2013-02-01,64
6,2013-06-01,54
7,2014-05-01,39
8,2016-05-01,38
9,2014-02-01,36


In [16]:
earliestDeliv['Date'].value_counts().reset_index()[0:10]

,index,Date
0,2013-01-01,1280
1,2015-01-01,481
2,2016-01-01,377
3,2014-01-01,265
4,2013-02-01,58
5,2015-02-01,49
6,2013-03-01,25
7,2015-05-01,17
8,2013-12-01,16
9,2014-02-01,13


In [17]:
earliestProd[earliestProd['Date']=="2016-01-01"][0:10]
# Ahwahnee Quik Mart:
#https://sdwis.waterboards.ca.gov/PDWW/JSP/ContactUs.jsp?tinwsys_is_number=7070&tinwsys_st_code=CA
# Activity Date = 11-06-2015

#Acton Four Square Church
# https://sdwis.waterboards.ca.gov/PDWW/JSP/WaterSystemDetail.jsp?tinwsys_is_number=2402&tinwsys_st_code=CA
#Activity Date = 08-08-2016
# There's a ocnsumer confidence report from April 13, 2015
# Found a report from 2003: https://merritt.cdlib.org/d/ark:%252F13030%252Fm5125t23/2/producer%252Fcadwsap-s1900894-001.pdf

# I doubt that tehese are 'new' water systems; the date just represents the first tiem the production data has been captured

,Water.System.Name,Date
2,141 SUBURBAN ROAD WATER SUPPLY,2016-01-01
8,5033 PENTECOST,2016-01-01
9,60TH STREET ASSOC. WATER SYSTEM,2016-01-01
11,7 LAKES RESORT,2016-01-01
31,ACTON FOUR SQUARE CHURCH,2016-01-01
47,AETNA SPRINGS GOLF COURSE,2016-01-01
54,AGUA DULCE WATER SYSTEM,2016-01-01
55,AGUA DULCE WINERY,2016-01-01
56,AGUA DULCE WOMEN'S CLUB,2016-01-01
58,AHWAHNEE QUIK MART,2016-01-01


###  Do systems report correctly, then report incorrectly later on?

In [18]:
earliest_correct = prod_processed[ (prod_processed['original_units_incorrect']==0)
                                 & (prod_processed['units_unavailable']==0) ][['Water.System.Name','Date']].groupby(["Water.System.Name"]).min().reset_index()
earliest_wrong = prod_processed[ (prod_processed['original_units_incorrect']==1)
                                & (prod_processed['units_unavailable']==0)][['Water.System.Name','Date']].groupby(["Water.System.Name"]).min().reset_index()

In [19]:
correctAndWrong = earliest_correct.merge(earliest_wrong,on="Water.System.Name",
                                        suffixes=['_correct','_wrong'],how="outer")

In [20]:
# systems wrong before correct
correctAndWrong[correctAndWrong['Date_wrong'] < correctAndWrong['Date_correct']].shape

(228, 3)

In [21]:
# systems wrong after correct
correctAndWrong[correctAndWrong['Date_wrong'] > correctAndWrong['Date_correct']].shape

(153, 3)

In [22]:
# systems never wrong
correctAndWrong[pd.isnull(correctAndWrong['Date_wrong'])].shape

(6179, 3)

In [23]:
# systems never correct
print(correctAndWrong[pd.isnull(correctAndWrong['Date_correct'])].shape)
correctAndWrong[pd.isnull(correctAndWrong['Date_correct'])][0:10]

(28, 3)


,Water.System.Name,Date_correct,Date_wrong
6561,AMERICAN RIVER COLLEGE,NaT,2016-01-01
6562,CAL TRANS-TIPTON NORTHBOUND REST STOP,NaT,2013-01-01
6563,CAMP CHILNUALNA WATER SYSTEM,NaT,2016-01-01
6564,CITY OF DORRIS,NaT,2013-01-01
6565,CITY OF LOYALTON,NaT,2014-01-01
6566,CITY OF WEED,NaT,2014-01-01
6567,CONSTELLATION WINE CO. MISSION BELL,NaT,2013-01-01
6568,DEETJEN S BIG SUR INN WS,NaT,2013-01-01
6569,"DORRIS, CITY OF",NaT,2016-01-01
6570,FOLSOM STATE PRISON,NaT,2013-01-01


In [24]:
correctAndWrong.shape

(6589, 3)

### How many records have unreported units?

In [25]:
# What % of records have units reported incorrectly?
prod_processed['units_unavailable'].value_counts() / prod_processed.shape[0]

0    0.997326
1    0.002674
Name: units_unavailable, dtype: float64

In [26]:
# What % of records have units reported incorrectly?
deliv_processed['units_unavailable'].value_counts() / prod_processed.shape[0]

0    0.365311
1    0.003712
Name: units_unavailable, dtype: float64

In [27]:
incorrectByYear = prod_processed[['Year','units_unavailable']].groupby('Year').agg({'units_unavailable':['sum','count']}).reset_index()
incorrectByYear['pct'] = incorrectByYear[('units_unavailable', 'sum')] / incorrectByYear[('units_unavailable', 'count')]
incorrectByYear

Year units_unavailable              pct
                      sum  count          
0  2013  171               43241  0.003955
1  2014  189               53338  0.003543
2  2015  119               54679  0.002176
3  2016  90                61566  0.001462

In [28]:
incorrectByYear = deliv_processed[['Year','units_unavailable']].groupby('Year').agg({'units_unavailable':['sum','count']}).reset_index()
incorrectByYear['pct'] = incorrectByYear[('units_unavailable', 'sum')] / incorrectByYear[('units_unavailable', 'count')]
incorrectByYear

Year units_unavailable              pct
                      sum  count          
0  2013  128               15622  0.008194
1  2014  98                17251  0.005681
2  2015  386               22158  0.017420
3  2016  178               23506  0.007573

### Export a file with incorrect/correct and available/unavailble units flags.

In [29]:
prod_processed['key'] = prod_processed[['Water.System.Name','Date']].apply(lambda x: "{}_{}".format(x[0],x[1]),axis=1)
deliv_processed['key'] = deliv_processed[['Water.System.Name','Date']].apply(lambda x: "{}_{}".format(x[0],x[1]),axis=1)

In [30]:
dupeKeyCt_deliv = deliv_processed['key'].value_counts().reset_index()
dupeKeyCt_deliv['key'].value_counts()

1    78489
2    24   
Name: key, dtype: int64

In [31]:
dupeKeyCt_prod = prod_processed['key'].value_counts().reset_index()
dupeKeyCt_prod['key'].value_counts()

1    212107
2    323   
4    7     
3    6     
5    5     
Name: key, dtype: int64

In [32]:
prod_processed[prod_processed['key'].isin(dupeKeyCt_prod[dupeKeyCt_prod['key']>1]['index'])==False].to_csv("{}2_explore_unit_correction___prod_before_unit_conversion.csv".format(processed_data_path),index=False)
deliv_processed[deliv_processed['key'].isin(dupeKeyCt_deliv[dupeKeyCt_deliv['key']>1]['index'])==False].to_csv("{}2_explore_unit_correction___deliv_before_unit_conversion.csv".format(processed_data_path),index=False)

In [33]:
prod_processed[prod_processed['key'].isin(dupeKeyCt_prod[dupeKeyCt_prod['key']>1]['index'])==False].shape
#212107

(212107, 32)

In [34]:
deliv_processed[deliv_processed['key'].isin(dupeKeyCt_prod[dupeKeyCt_prod['key']>1]['index'])==False].shape
#78393, 34)

(78393, 34)